In [2]:
import sys, os, csv, subprocess
from datetime import date
from io import StringIO
import pandas as pd, numpy as np
import time,timeit,numba,cython
from scipy.spatial import distance as sp_distance
from scipy import stats as sp_stats
%load_ext Cython

In [7]:
test = pd.read_csv('bruh.csv', sep=',', index_col=[0,1,2,3])
refDf = pd.read_csv('data/AllReferenceExp_20191018_commonFeaturesOrder.csv', sep=',', index_col=[0,1,2,3])
lol = pd.read_csv('bruh2.csv', sep=',', index_col=[0,1,2,3], header=[0,1,2,3])
# display(test.head(5))
# display(refDf.head(5))
display(lol.head(5))

,,,CompoundName,SW218124-2,SW219896-1,SW197639-3,SW218086-2,SW219171-1,SW219073-1,SW222338-1,SW219934-1,SW218112-2,SW196399-3,...,SW201532,SW202321,SW200801,SW201563,SW202283,SW200864,SW201529,SW200821,SW218625,SW203104
,,,Concentration,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
,,,PlateName,SP0135,SP0135,SP0135,SP0135,SP0135,SP0135,SP0135,SP0135,SP0135,SP0135,...,SP20265,SP20265,SP20265,SP20265,SP20265,SP20265,SP20265,SP20265,SP20265,SP20265
,,,WellID,E03,O15,N08,G22,J03,M07,B04,E18,C17,F05,...,C18,M07,B07,F19,K09,H10,C15,D07,H01,P19
CompoundName,Concentration,PlateName,WellID,,,,,,,,,,,,,,,,,,,,,
SW219778-1,2.0,SP0142,L08,-0.125537,-0.247795,0.372787,-0.356639,-0.401852,-0.476957,-0.445545,-0.363545,-0.103664,-0.488742,...,0.435414,0.456260,0.481989,0.417482,-0.318081,0.596347,0.322328,0.339201,0.296740,-0.437656
SW219366-1,2.0,SP0142,O19,0.016927,-0.024736,0.268461,-0.220774,-0.330168,-0.239338,0.041293,-0.337630,0.191660,-0.177525,...,0.083324,0.241314,0.343155,0.105729,0.085858,0.245149,-0.016376,0.074590,0.257471,-0.099798
SW218151-2,2.0,SP0142,I04,-0.331452,-0.511265,0.354381,-0.513809,-0.284220,-0.291399,-0.543666,-0.415235,-0.146666,-0.389171,...,0.337200,0.508865,0.312689,0.277765,-0.533107,0.405282,0.273746,0.325676,0.129554,-0.525544
SW219425-1,2.0,SP0142,K11,0.095889,-0.294668,-0.153401,-0.122234,0.177280,0.060252,-0.250991,0.150090,-0.274022,0.128566,...,0.125157,0.056578,-0.144337,-0.048812,-0.247570,-0.047608,0.183385,-0.047499,-0.063099,-0.072442
SW219079-1,2.0,SP0142,J06,0.165803,0.245081,-0.253436,0.200482,0.376068,0.564539,0.710451,0.291160,0.333409,0.486894,...,-0.404000,-0.398649,-0.345960,-0.392826,0.666163,-0.486151,-0.377833,-0.251702,-0.202090,0.673988


In [146]:
@numba.jit(nogil=True)
def correlation(u, v, w=None, centered=True): #rip from scipy.spatial.distance source
    """
    Compute the correlation distance between two 1-D arrays.
    The correlation distance between `u` and `v`, is
    defined as
    .. math::
        1 - \\frac{(u - \\bar{u}) \\cdot (v - \\bar{v})}
                  {{||(u - \\bar{u})||}_2 {||(v - \\bar{v})||}_2}
    where :math:`\\bar{u}` is the mean of the elements of `u`
    and :math:`x \\cdot y` is the dot product of :math:`x` and :math:`y`.
    Parameters
    ----------
    u : (N,) array_like
        Input array.
    v : (N,) array_like
        Input array.
    w : (N,) array_like, optional
        The weights for each value in `u` and `v`. Default is None,
        which gives each value a weight of 1.0
    Returns
    -------
    correlation : double
        The correlation distance between 1-D array `u` and `v`.
    """
#     u = _validate_vector(u)
#     v = _validate_vector(v)
#     if w is not None:
#         w = _validate_weights(w)
    if centered:
        umu = np.mean(u) #, weights=w)
        vmu = np.mean(v) #, weights=w)
        u = u - umu
        v = v - vmu
    uv = np.mean(u * v) #, weights=w)
    uu = np.mean(np.square(u))#, weights=w)
    vv = np.mean(np.square(v))#, weights=w)
    dist = 1.0 - uv / np.sqrt(uu * vv)
    return dist

@numba.njit(parallel=False,nogil=True)
def numbaParallelCorrDistonRef(compSig,refArray):
    """attempt at making a nice numba accelerated version of CorrDist function,
    by parallelizing the specific compDist calculation to refSigs."""
#     if isinstance(refArray, pd.DataFrame):
#         refArray = refArray.to_numpy()
    numRefSigs = refArray.shape[0]
    returnDists = np.zeros(refArray.shape)
    for i in numba.prange(numRefSigs):
        returnDists[i] = (1.0 - np.mean(compSig-np.mean(compSig) * refArray[i]-np.mean(refArray[i])) / np.sqrt(np.mean(np.square(compSig-np.mean(compSig))) * np.mean(np.square(refArray[i]-np.mean(refArray[i])))))
#         correlation(compSig, refArray[i])
    return returnDists

/tmp/ipykernel_286/1194801673.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nogil=True)


In [200]:
# %%cython
from scipy.spatial import distance as sp_distance
from scipy import stats as sp_stats
import numpy as np
#     @numba.jit#(nogil=True)
def corrDist (compSig:np.ndarray, refArray:np.ndarray) -> np.ndarray:
    returnDists = [0]*refArray.shape[0]
    for i in range(refArray.shape[0]):
        x = refArray[i,:]
        returnDists[i] = sp_distance.correlation(compSig, x)
    return np.array(returnDists)

#     @numba.jit#(nogil=True)
def pearsonr (compSig:np.ndarray, refArray:np.ndarray) -> np.ndarray:
    returnCorrs = [0]*refArray.shape[0]
    for i in range(refArray.shape[0]):
        x = refArray[i,:]
        returnCorrs[i] = sp_stats.pearsonr(x, compSig)[0]
    return np.array(returnCorrs)


In [211]:
## TESTING pearsonR
allDF = pd.read_csv("data/AllReferenceExp_20191018_commonFeaturesOrder.csv",index_col=[0,1,2,3])
testDF = pd.read_csv("data/SP0142_20171024_HeLa_10x_0_CP_histdiff_Concatenated.csv",index_col=[0,1,2,3])
testDF = testDF.reindex(columns = allDF.columns)
testSig_comp = testDF.index[0]
testSig = testDF.loc[testSig_comp]

# %timeit print(pearsonr(testSig.values, allDF.to_numpy()))
# 2.78 s ± 36.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

@numba.jit(nopython=True, parallel=True)
def pearsonr(compSig, refArray):
    num_ref_sigs = refArray.shape[0]
    return_corrs = np.empty(num_ref_sigs)
    
    for i in numba.prange(num_ref_sigs):
        x = refArray[i, :]
        return_corrs[i] = np.corrcoef(x, compSig)[0, 1]
    
    return return_corrs

%timeit print(pearsonr(testSig.values, allDF.to_numpy()))
# JIT:
# 16.1 ms ± 446 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
# with parallelization:
# 5.54 ms ± 737 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)

[-0.12553707 -0.24779484  0.37278739 ...  0.33920066  0.2967402
 -0.43765571]
[-0.12553707 -0.24779484  0.37278739 ...  0.33920066  0.2967402
 -0.43765571]
[-0.12553707 -0.24779484  0.37278739 ...  0.33920066  0.2967402
 -0.43765571]
[-0.12553707 -0.24779484  0.37278739 ...  0.33920066  0.2967402
 -0.43765571]
[-0.12553707 -0.24779484  0.37278739 ...  0.33920066  0.2967402
 -0.43765571]
[-0.12553707 -0.24779484  0.37278739 ...  0.33920066  0.2967402
 -0.43765571]
[-0.12553707 -0.24779484  0.37278739 ...  0.33920066  0.2967402
 -0.43765571]
[-0.12553707 -0.24779484  0.37278739 ...  0.33920066  0.2967402
 -0.43765571]
17.8 ms ± 2.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [136]:
def testFingerprintPrcess(exp_df,ref_df):
    from scipy.spatial import distance as sp_distance
    from scipy import stats as sp_stats
    #import numpy as np
    # @numba.jit#(nogil=True)
    def corrDist (compSig:np.ndarray, refArray:np.ndarray) -> np.ndarray:
        returnDists = [0]*refArray.shape[0]
        for i in range(refArray.shape[0]):
            x = refArray[i,:]
            returnDists[i] = sp_distance.correlation(compSig, x)
        return np.array(returnDists)

    # @numba.jit#(nogil=True)
    def pearsonr (compSig:np.ndarray, refArray:np.ndarray) -> np.ndarray:
        returnCorrs = [0]*refArray.shape[0]
        for i in range(refArray.shape[0]):
            x = refArray[i,:]
            returnCorrs[i] = sp_stats.pearsonr(x, compSig)[0]
        return np.array(returnCorrs)
    
    

#     def nestedApply_onExpDF (compSig):
#         distance = ref_df.apply(lambda x: corrDist(compSig, x.values), axis=1)
#         similarity = ref_df.apply(lambda x: pearsonr(compSig,x.values), axis=1)
#         return distance, similarity
    
    reporting_df = pd.DataFrame(columns=["TopSimilarRefComps","CorrespPearsonSim","CorrespCorrDistance"], index=exp_df.index)
    # one_to_one = list() # for sanity checks
    refcompounds = ["._.".join([str(y) for y in x]) for x in ref_df.index]
    print(ref_df.to_numpy().shape)
    
#     distance,similarity =
    distance = exp_df.apply(lambda compSig: corrDist(compSig.to_numpy(),ref_df.to_numpy()),axis=1)
    similarity = exp_df.apply(lambda compSig: pearsonr(compSig.to_numpy(),ref_df.to_numpy()),axis=1)
    reporting_df['TopSimilarRefComps'] = ".__.".join(refcompounds)
    reporting_df['CorrespCorrDistance'] = \
            [".__.".join(["{:.10f}".format(d_1) for d_1 in d]) for d in distance]
    reporting_df['CorrespPearsonSim'] = \
            [".__.".join(["{:.10f}".format(s_1) for s_1 in s]) for s in similarity]
    
    
#     for compSig in exp_df.iterrows():

#         comp, compSig = compSig.Index, np.array(compSig[1:])
#         print(compSig)
#         comp, compSig = compSig[0], compSig[1].values
    tester = pd.DataFrame(sp_distance.cdist(ref_df.to_numpy(), exp_df.to_numpy(),metric='correlation'),index=ref_df.index,columns=exp_df.index)
    tester2 = pd.DataFrame({distance.index[i]:d for i,d in enumerate(distance)})
    print(len(distance),tester,tester2,sep="\n\n")
#           ,pd.DataFrame({i:s for i,s in enumerate(similarity)}),sep="\n\n")
    testOrient = (tester.to_numpy() == np.transpose(np.array([d for d in distance])))
    print(testOrient,tester.shape,np.transpose(np.array([d for d in distance])).shape,np.sum(testOrient))
#           ,pd.DataFrame({i:s for i,s in enumerate(similarity)}).T,sep="\n\n")
        
    #convert to pd.Series for easier sorting and indexing
#     similarity_1 = [pd.Series({c:"{:.10f}".format(s) for s, c in zip(s_1,refcompounds)}).sort_values(ascending =False) for s_1 in similarity]
#     distance_1 = [pd.Series({c:"{:.10f}".format(d) for d, c in zip(d_1,refcompounds)}).sort_values() for d_1 in distance]
#     comp_1 = "._.".join([str(y) for y in comp]) #needed to access later for one_to_one
#     for i in range(exp_df.shape[0]):
#         print(similarity_1[i].index == distance_1[i].index,(similarity_1[i].index == distance_1[i].index).sum())
        
#     #print(comp,comp_1,similarity_1.index[0],distance_1.index[0],sep="\n")    
#     [print(ref,s) for ref,s in zip(similarity.index,similarity_1)]

#     # we should verify that the all the similairies and distances are in the same order
#     # otherwise the refCompound column cannot apply to both.
#     if (similarity_1.index == distance_1.index).sum():
#         #save the sort_value index order and reindex for performance
#         refcompounds_1 = distance_1.index.tolist()
#         similarity_1.reindex(index = refcompounds)
#         distance_1.reindex(index = refcompounds)

#         # print("reindexed the index")

#         reporting_df.loc[comp,"TopSimilarRefComps"] = ".__.".join(refcompounds_1[1:])
#         reporting_df.loc[comp,'CorrespPearsonSim'] = ".__.".join(\
#             ["{:.10f}".format(s) for s in similarity_1.drop(index=comp_1)])
#         reporting_df.loc[comp,"CorrespCorrDistance"] = ".__.".join(\
#             ["{:.10f}".format(d) for d in distance_1.drop(index=comp_1)])
#         # one_to_one.append((similarity_1[comp_1],distance_1[comp_1]))
#     else:
#         reporting_df.loc[comp,"TopSimilarRefComps"] = ".__.".join(refcompounds)
#         reporting_df.loc[comp,'CorrespPearsonSim'] = ".__.".join(\
#             ["{:.10f}".format(s) for s in similarity])
#         reporting_df.loc[comp,"CorrespCorrDistance"] = ".__.".join(\
#             ["{:.10f}".format(d) for d in distance])
#         # one_to_one.append((similarity[comp],distance[comp]))
# #         print(time.clock(),t2-t)
#     break

    return reporting_df#, one_to_one #for sanity checks

In [5]:
allDF = pd.read_csv("data/AllReferenceExp_20191018_commonFeaturesOrder.csv",index_col=[0,1,2,3])
testDF = pd.read_csv("data/SP0142_20171024_HeLa_10x_0_CP_histdiff_Concatenated.csv",index_col=[0,1,2,3])
testDF = testDF.reindex(columns = allDF.columns)
pd.__version__
allDF.to_numpy().shape

display(allDF.head(5))
display(testDF.head(5))

,,,,Cell_Number_of_Micronuclei_Micronuclei_EdU,Cell_Number_of_Micronuclei_Micronuclei_cyto,Cell_PH3_Nucleus_Integr_Intensity_MultiWaveScoring_EdU,Cell_PH3_Stained_Integr_Intensity_MultiWaveScoring_EdU,Cell_Pit_Integrated_Intensity_Transfluor_cyto,Cell_Pit_Total_Area_Transfluor_cyto,Pit_Area_Per_Cell_Transfluor_cyto,Pit_Count_Per_Cell_Transfluor_cyto,Cell_DNA_Area_Micronuclei_EdU,Cell_Nuclear_Integrated_Intensity_Micronuclei_EdU,...,Cell_Pit_Count_Transfluor_EdU,Cell_Pit_Integrated_Intensity_Transfluor_EdU,Pit_Area_Per_Cell_Transfluor_EdU,Pit_Count_Per_Cell_Transfluor_EdU,Negative_EdU_Mean_Stain_Area_MultiWaveScoring_EdU,Negative_EdU_Mean_Stain_Integr_Intens_MultiWaveScoring_EdU,Pct_Mononucleated_Cells_Micronuclei_EdU,Pct_Mononucleated_Cells_Micronuclei_cyto,Shape_factor_IMA_Summary_EdU,Shape_factor_IMA_Summary_cyto
CompoundName,Concentration,PlateName,WellID,,,,,,,,,,,,,,,,,,,,,
SW218124-2,10.0,SP0135,E03,0.000045,-2.623516e-09,-4.513278e-09,-5.364705e-09,4.293329e-06,-8.943764e-07,-0.000633,-0.000206,3.946115e-07,0.0,...,0.0,-5.319644e-09,0.0,0.0,0.87500,0.0,-0.624969,-0.547401,-0.346275,-0.590904
SW219896-1,10.0,SP0135,O15,0.002279,-2.623516e-09,-4.513278e-09,-5.364705e-09,1.769610e-05,2.705429e-04,0.052391,0.085167,2.327163e-07,0.0,...,0.0,-5.319644e-09,0.0,0.0,0.37500,0.0,-0.295868,-0.158730,-0.346275,-0.470787
SW197639-3,10.0,SP0135,N08,-0.000371,-2.623516e-09,-4.513278e-09,-5.364705e-09,2.692871e-06,5.437617e-06,-0.000633,-0.000206,-6.433700e-10,0.0,...,0.0,5.640772e-09,0.0,0.0,0.00000,0.0,-0.029755,-0.038612,0.000572,0.000572
SW218086-2,10.0,SP0135,G22,-0.000013,-2.623516e-09,-4.513278e-09,-5.364705e-09,8.288334e-07,-9.748516e-06,-0.000633,-0.000206,-6.433700e-10,0.0,...,0.0,-5.319644e-09,0.0,0.0,0.00000,0.0,-0.010223,0.095741,0.000572,-0.141197
SW219171-1,10.0,SP0135,J03,-0.000072,-2.623516e-09,-4.513278e-09,-5.364705e-09,2.907109e-06,-1.299095e-05,-0.000633,0.085167,6.103021e-08,0.0,...,0.0,-5.319644e-09,0.0,0.0,0.09375,0.0,0.349091,0.261269,0.000572,0.000572


,,,,Cell_Number_of_Micronuclei_Micronuclei_EdU,Cell_Number_of_Micronuclei_Micronuclei_cyto,Cell_PH3_Nucleus_Integr_Intensity_MultiWaveScoring_EdU,Cell_PH3_Stained_Integr_Intensity_MultiWaveScoring_EdU,Cell_Pit_Integrated_Intensity_Transfluor_cyto,Cell_Pit_Total_Area_Transfluor_cyto,Pit_Area_Per_Cell_Transfluor_cyto,Pit_Count_Per_Cell_Transfluor_cyto,Cell_DNA_Area_Micronuclei_EdU,Cell_Nuclear_Integrated_Intensity_Micronuclei_EdU,...,Cell_Pit_Count_Transfluor_EdU,Cell_Pit_Integrated_Intensity_Transfluor_EdU,Pit_Area_Per_Cell_Transfluor_EdU,Pit_Count_Per_Cell_Transfluor_EdU,Negative_EdU_Mean_Stain_Area_MultiWaveScoring_EdU,Negative_EdU_Mean_Stain_Integr_Intens_MultiWaveScoring_EdU,Pct_Mononucleated_Cells_Micronuclei_EdU,Pct_Mononucleated_Cells_Micronuclei_cyto,Shape_factor_IMA_Summary_EdU,Shape_factor_IMA_Summary_cyto
CompoundName,Concentration,PlateName,WellID,,,,,,,,,,,,,,,,,,,,,
SW219778-1,2.0,SP0142,L08,-0.000020,-0.000195,-0.002018,-0.000003,-0.000002,-2.816838e-07,-0.000656,-0.004524,-4.971930e-11,-2.549183e-10,...,0.000103,0.000481,0.003754,-0.005699,-0.000206,0.0000,-0.107544,-0.116447,-0.147301,0.090462
SW219366-1,2.0,SP0142,O19,-0.000002,0.000015,-0.002067,-0.000004,-0.000031,-3.742161e-05,-0.000656,0.057259,1.432477e-07,-2.549183e-10,...,-0.001206,-0.000646,-0.064301,-0.093590,-0.000206,0.0000,-0.413696,-0.127678,-0.147301,0.090462
SW218151-2,2.0,SP0142,I04,-0.000050,-0.000004,-0.002052,-0.000003,-0.000014,-4.355672e-05,-0.000656,-0.004524,-2.926850e-08,-2.549183e-10,...,0.000082,0.002140,0.003754,-0.005699,-0.000206,0.0000,-0.120728,0.080315,-0.147301,0.513313
SW219425-1,2.0,SP0142,K11,-0.000058,0.000041,-0.000976,0.000096,0.011367,9.338299e-04,0.196945,-0.004524,-2.926850e-08,-2.549183e-10,...,0.113682,0.320233,0.568207,0.549644,0.985558,0.9375,0.457642,0.198967,0.368980,0.351204
SW219079-1,2.0,SP0142,J06,-0.000002,0.008782,-0.002019,-0.000003,-0.000005,3.272586e-04,0.080246,0.297493,-2.926850e-08,-2.549183e-10,...,-0.005480,-0.005224,-0.312347,-0.093590,-0.000206,0.0000,0.340942,0.564201,-0.018394,0.314095


In [107]:
# os.path.abspath('.')
# os.chdir("../../../Scripts_AL/")

In [9]:
from scipy.spatial import distance as sp_distance
from scipy import stats as sp_stats

In [25]:
def numbaParallelPearsonR(compSig: np.ndarray, refArray: np.ndarray) -> np.ndarray:
    returnCorrs = np.zeros(refArray.shape[0])
    for i in range(refArray.shape[0]):
        x = refArray[i,:]
        returnCorrs[i] = sp_stats.pearsonr(x, compSig)[0]
        # print(returnCorrs[i], file=sys.stderr)
    return returnCorrs

In [179]:
def corrDist (compSig:np.ndarray, refArray:np.ndarray) -> np.ndarray:
    returnDists = [0]*refArray.shape[0]
    for i in range(refArray.shape[0]):
        x = refArray[i,:]
        returnDists[i] = sp_distance.correlation(compSig, x)
    return np.array(returnDists)

In [209]:
@numba.jit(nopython=True, parallel=True)
def corrDist(compSig, refArray):
    def cor(u,v, centered=True):
        if centered:
            umu = np.average(u)
            vmu = np.average(v)
            u = u - umu
            v = v - vmu
        uv = np.average(u*v)
        uu = np.average(np.square(u))
        vv = np.average(np.square(v))
        dist = 1 - uv / np.sqrt(uu*vv)
        return np.abs(dist)

    num_ref_sigs = refArray.shape[0]
    return_dists = np.empty(num_ref_sigs)
    
    for i in numba.prange(num_ref_sigs):
        x = refArray[i]
        return_dists[i] = cor(compSig, x)
        # return_dists[i] = np.sum((compSig - x)**2) / (np.sqrt(np.sum(compSig**2)) * np.sqrt(np.sum(x**2)))
    
    return return_dists

testSig_comp = testDF.index[0]
testSig = testDF.loc[testSig_comp]
#print(testSig.to_numpy().shape[0])
# print(len(allDF.iloc[0].to_numpy()))
# %timeit print(np.array([(sp_distance.correlation(testSig.to_numpy() , allDF.iloc[i].to_numpy()),\
#                         sp_stats.pearsonr(testSig.to_numpy(), allDF.iloc[0].to_numpy()))\
#                         for i in range(testSig.to_numpy().shape[0])]))

# %timeit print(numbaParallelCorrDistonRef(testSig.values, allDF.to_numpy())[:,0])#,\
# 14.5 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
# sp_stats.pearsonr(testSig.values,allDF.to))

%timeit print(corrDist(testSig.values, allDF.to_numpy()))
# 177 ms ± 4.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# using the above function:
#   7.57 ms ± 220 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
# using np.prange:
#   2.06 ms ± 1.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

[1.12553707 1.24779484 0.62721261 ... 0.66079934 0.7032598  1.43765571]
[1.12553707 1.24779484 0.62721261 ... 0.66079934 0.7032598  1.43765571]
[1.12553707 1.24779484 0.62721261 ... 0.66079934 0.7032598  1.43765571]
[1.12553707 1.24779484 0.62721261 ... 0.66079934 0.7032598  1.43765571]
[1.12553707 1.24779484 0.62721261 ... 0.66079934 0.7032598  1.43765571]
[1.12553707 1.24779484 0.62721261 ... 0.66079934 0.7032598  1.43765571]
[1.12553707 1.24779484 0.62721261 ... 0.66079934 0.7032598  1.43765571]
[1.12553707 1.24779484 0.62721261 ... 0.66079934 0.7032598  1.43765571]
2.4 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [137]:
fingerprintComparison = testFingerprintPrcess(testDF,allDF)
# %prun -l 10 -s cumulative testFingerprintPrcess(testDF,allDF)
# %timeit testFingerprintPrcess(testDF,allDF)
# , sameComp 
# with jit: 6min 11s ± 37.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
# without jit: 5min 12s ± 17.9 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
# without extra sorting/ordering: 5min 4s ± 21.1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
# pull corrDist and pearsonR out of cell (maybe cython): 5min 37s ± 33.6 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

/tmp/ipykernel_286/3013605516.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit#(nogil=True)
/tmp/ipykernel_286/3013605516.py:13: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit#(nogil=True)
/tmp/ipykernel_286/3013605516.py:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "corrDist" fail

(4738, 245)


/home/derfelt/.local/lib/python3.11/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "corrDist" was compiled in object mode without forceobj=True.

File "../../../tmp/ipykernel_286/3013605516.py", line 8:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
/home/derfelt/.local/lib/python3.11/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected. This is deprecated behaviour that will be removed in Numba 0.59.0.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "../../../tmp/ipykernel_286/3013605516.py", line 8:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaDeprecationWarning(msg,
/tmp/ipykernel_286/3013605516.py:13: NumbaWarning: 
Compilation is falling back to object mode WITH lo

KeyboardInterrupt: 

In [16]:
print(fingerprintComparison)
fingerprintComparison.to_excel("SP0142_20171024_HeLa_10x_0_CP_FeatureReport.xlsx",engine='xlsxwriter')


                                                                            TopSimilarRefComps  \
CompoundName Concentration PlateName WellID                                                      
SW219778-1   2.0           SP0142    L08     SW218124-2._.10.0._.SP0135._.E03.__.SW219896-1...   
SW219366-1   2.0           SP0142    O19     SW218124-2._.10.0._.SP0135._.E03.__.SW219896-1...   
SW218151-2   2.0           SP0142    I04     SW218124-2._.10.0._.SP0135._.E03.__.SW219896-1...   
SW219425-1   2.0           SP0142    K11     SW218124-2._.10.0._.SP0135._.E03.__.SW219896-1...   
SW219079-1   2.0           SP0142    J06     SW218124-2._.10.0._.SP0135._.E03.__.SW219896-1...   
...                                                                                        ...   
SW219664-1   2.0           SP0142    I09     SW218124-2._.10.0._.SP0135._.E03.__.SW219896-1...   
SW197554-3   2.0           SP0142    J16     SW218124-2._.10.0._.SP0135._.E03.__.SW219896-1...   
SW219415-1   2.0    

ModuleNotFoundError: No module named 'xlsxwriter'

In [70]:
pd.DataFrame(sameComp,columns=['similarity','distance'],index=testDF.index)

,,,,similarity,distance
CompoundName,Concentration,PlateName,WellID,,
SW219778-1,2.0,SP0142,L08,1.0,0.0
SW219366-1,2.0,SP0142,O19,1.0,0.0
SW218151-2,2.0,SP0142,I04,1.0,0.0
SW219425-1,2.0,SP0142,K11,1.0,0.0
SW219079-1,2.0,SP0142,J06,1.0,0.0
SW222338-1,2.0,SP0142,B04,1.0,0.0
SW219396-1,2.0,SP0142,J05,1.0,0.0
SW196607-4,2.0,SP0142,G20,1.0,0.0
SW219879-1,2.0,SP0142,H11,1.0,0.0


In [41]:
v = [-1, 5, 0, 0, 10, 0, -7]
v1 = [1, 0, 0, 0, 0, 0, 0]
v2 = [0, 1, 0, 0, 1, 0, 0]
v3 = [1, 1, 0, 0, 0, 0, 1]    
df = pd.DataFrame([v1, v2, v3], columns=['a', 'b', 'c', 'd', 'e', 'f', 'g'])
from scipy.stats.stats import pearsonr
print(df)
s_corrs = df.apply(lambda x: pearsonr(x.values, v)[0], axis=1)
s_corrs

   a  b  c  d  e  f  g
0  1  0  0  0  0  0  0
1  0  1  0  0  1  0  0
2  1  1  0  0  0  0  1


0   -0.166667
1    0.839146
2   -0.353553
dtype: float64

### Similarity Report for Tannia's RAW cell data
apply the similarity report function created above, to generate reports for the NPFs against the rest in the individual datasets

In [74]:
os.chdir('../HiFAN/Tannia/RAW264cells_SChem/')

In [86]:
DMSO_df = pd.read_csv("RAW264Cells_FeaturesReduced/RAW264cells_HiFANnoCts_DMSO_reduced.csv",index_col=0)
DMSO_LPS_df = pd.read_csv("RAW264Cells_FeaturesReduced/RAW264cells_HiFANnoCts_DMSO+LPS_reduced.csv",index_col=0)
LPS_df = pd.read_csv("RAW264Cells_FeaturesReduced/RAW264cells_HiFANnoCts_LPS_reduced.csv",index_col=0)

In [91]:
DMSO_similiarityReport = testFingerprintPrcess(DMSO_df.loc[[x for x in DMSO_df.index if x.startswith("SP20265") or x.startswith("SP20264")]],DMSO_df)
DMSO_LPS_similarityReport = testFingerprintPrcess(DMSO_LPS_df.loc[[x for x in DMSO_LPS_df.index if x.startswith("SP20265") or x.startswith("SP20264")]],DMSO_LPS_df)
LPS_similarityReport = testFingerprintPrcess(LPS_df.loc[[x for x in LPS_df.index if x.startswith("SP20265") or x.startswith("SP20264")]],LPS_df)

In [101]:
DMSO_LPS_similarityReport.CorrespCorrDistance


#  print(DMSO_df.loc[[x for x in DMSO_df.index if x.startswith("SP20265") or x.startswith("SP20264")]].shape,\
#       DMSO_LPS_df.loc[[x for x in DMSO_LPS_df.index if x.startswith("SP20265") or x.startswith("SP20264")]].shape,\
#       LPS_df.loc[[x for x in LPS_df.index if x.startswith("SP20265") or x.startswith("SP20264")]].shape)


Features
SP20264._.DMSO_A01_-1.0    0.0458660910.__.0.0634208545.__.0.0809264832._...
SP20264._.DMSO_A02_-1.0    0.0814656154.__.0.0830850453.__.0.0870765451._...
SP20264._.DMSO_B01_-1.0    0.0491528696.__.0.0781651371.__.0.0968852320._...
SP20264._.DMSO_B02_-1.0    0.0623305403.__.0.1023847599.__.0.1239171461._...
SP20264._.DMSO_C01_-1.0    0.0567753436.__.0.0628449403.__.0.0855086347._...
SP20264._.DMSO_C02_-1.0    0.0618566780.__.0.0943016206.__.0.1079366243._...
SP20264._.DMSO_D01_-1.0    0.0634208545.__.0.0788555390.__.0.0805071849._...
SP20264._.DMSO_D02_-1.0    0.0448119346.__.0.0623305403.__.0.1037767343._...
SP20264._.DMSO_E01_-1.0    0.0491826965.__.0.1296133211.__.0.1311895735._...
SP20264._.DMSO_E02_-1.0    0.0628449403.__.0.0921236986.__.0.1016477370._...
SP20264._.DMSO_F01_-1.0    0.0955272176.__.0.1602466714.__.0.1652536607._...
SP20264._.DMSO_F02_-1.0    0.0448119346.__.0.1023847599.__.0.1374846465._...
SP20264._.DMSO_G01_-1.0    0.0491826965.__.0.1381643761.__.0.153196

In [102]:
DMSO_similiarityReport.to_excel("RAW264Cells_FeaturesReduced/RAW264Cells_HiFANnoCts_DMSO_NPFsimilarityReport.xlsx")
DMSO_LPS_similarityReport.to_excel("RAW264Cells_FeaturesReduced/RAW264Cells_HiFANnoCts_DMSO+LPS_NPFsimilarityReport.xlsx")
LPS_similarityReport.to_excel("RAW264Cells_FeaturesReduced/RAW264Cells_HiFANnoCts_LPS_NPFsimilarityReport.xlsx")